In [2]:
import tensorflow.compat.v2 as tf

2024-12-03 19:07:10.757564: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-03 19:07:10.775193: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733224030.794366 2096427 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733224030.800238 2096427 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 19:07:10.821536: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
loaded_model = tf.saved_model.load('/home/douliyang/large/mlir-workspace/iree-tutorial/model/mobilenet-v2')
call = loaded_model.__call__.get_concrete_function(
         tf.TensorSpec([1, 224, 224, 3], tf.float32))
signatures = {'predict': call}
tf.saved_model.save(loaded_model,
  '/home/douliyang/large/mlir-workspace/iree-tutorial/model/mobilenet-v2', signatures=signatures)

INFO:tensorflow:Assets written to: /home/douliyang/large/mlir-workspace/iree-tutorial/model/mobilenet-v2/assets


INFO:tensorflow:Assets written to: /home/douliyang/large/mlir-workspace/iree-tutorial/model/mobilenet-v2/assets


In [5]:
loaded_model = tf.saved_model.load('/home/douliyang/large/mlir-workspace/iree-tutorial/model/mobilenet-v2')
print(list(loaded_model.signatures.keys()))

['predict']


In [6]:
!iree-import-tf \
  --tf-import-type=savedmodel_v1 \
  --tf-savedmodel-exported-names=predict \
  /home/douliyang/large/mlir-workspace/iree-tutorial/model/mobilenet-v2 -o iree_input.mlir

2024-12-03 19:08:57.044138: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-03 19:08:57.059416: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733224137.077039 2102121 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733224137.082107 2102121 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 19:08:57.100642: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [8]:
!iree-compile \
    --iree-hal-target-backends=cuda \
    --iree-cuda-target=sm_86 \
    iree_input.mlir -o mobilenet_cuda.vmfb

In [9]:
!iree-run-module \
    --device=cuda \
    --module=mobilenet_cuda.vmfb \
    --function=predict \
    --input="1x224x224x3xf32=0"

EXEC @predict
result[0]: hal.buffer_view
1x1001xf32=[-0.190896 -0.368495 0.791366 -0.536175 0.836177 2.03093 0.444975 0.633471 -0.52778 -0.220641 -0.284229 -0.575583 -0.75141 -0.864841 -1.04724 -0.14838 -0.5843 0.802287 -0.587424 -0.302664 -1.79424 -1.12465 1.87333 2.91976 1.25541 -0.210684 -0.250524 -0.562352 -1.60437 -0.23303 0.325934 0.724321 1.30018 0.103095 -0.213149 1.96294 -1.45143 -0.187634 -0.609063 -0.168662 -1.18992 -0.608314 -0.386122 -0.0878132 0.734984 -0.643625 -0.887488 -0.624094 -1.48707 -0.399353 -0.226703 0.748592 -0.136391 1.01801 0.778209 1.5221 -1.43791 -1.03928 -0.174579 0.219547 0.76114 1.76436 -0.125505 0.64919 -0.294448 -0.0183733 0.24737 -0.271909 -1.10309 0.929539 1.3333 -0.0224847 1.13154 -0.579985 0.800653 -1.47004 1.11697 0.236539 -0.00632849 0.772746 -0.844061 0.865322 -1.19403 0.346387 -0.229971 -1.19672 -0.252986 -0.883019 -1.21736 -0.620748 -0.0770344 -1.26719 -0.0454472 -0.41274 -0.83527 -1.14898 -0.939296 -0.859859 -1.02453 0.0517 1.49961 0.592278 1